In [ ]:
import comet_ml
import os
import json
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import keras
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from ml_util.monitor.comet_ml_exp import initialize_comet_ml_experiment
from basic_transformer.models.basic_transformer import BasicTransformer
from basic_transformer import utils as local_util
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

In [ ]:
DATASET = 'yelp'  # 'yelp' or 'imdb'
DIM = 128
NUM_WORDS = 5_000
TEXT_COLUMN = 'review'
LABEL_COLUMN = 'sentiment'
LABEL_MAPPING = {'negative': 0, 'positive': 1}
MAX_SEQ_LEN = 128
# dataloaders
BATCH_SIZE = 16
SHUFFLE = True
DEBUG_DF = False

In [ ]:
# get model
model = BasicTransformer(dim=DIM, num_embeddings=NUM_WORDS, embedding_dim=DIM)

In [ ]:
if DATASET == 'yelp':
    df = pd.read_csv("/media/can/MyData/datasets/yelp/df.csv")
    df['sentiment'] = df['stars'].replace({1: 'negative', 5: 'positive'})
    df = df.rename(columns={'text': 'review'})
    df = df.sample(n=50_000, random_state=RANDOM_SEED)
elif DATASET == 'imdb':
    df = pd.read_csv("/media/can/MyData/datasets/imdb-50k-movie-review/IMDB Dataset.csv")
else:
    raise ValueError("Invalid data: {}".format(str(DATASET)))

In [ ]:
txts_positive = df[df['sentiment'] == 'positive']['review'].sample(3).tolist()
txts_negative = df[df['sentiment'] == 'negative']['review'].sample(3).tolist()

In [ ]:
lens = df['review'].apply(lambda x: len(x.split(' ')))
lens.mean()

In [ ]:
# generate a fake df for debugging
if DEBUG_DF:
    n_positive, n_negative = 5000, 5000
    positive_label, negative_label = 'positive', 'negative'
    positive_text = ' '.join(['good'] * 10)
    negative_text = ' '.join(['bad'] * 10)
    df = [(positive_text, positive_label)] * n_positive + [(negative_text, negative_label)] * n_negative
    df = pd.DataFrame(df)
    df.columns = ('review', 'sentiment')
    df

In [ ]:
datagen = local_util.dataset_generator.TextDataset(df=df, 
                                                   num_words=NUM_WORDS, 
                                                   text_column=TEXT_COLUMN, 
                                                   label_column=LABEL_COLUMN, 
                                                   label_mapping=LABEL_MAPPING, 
                                                   max_seq_len=MAX_SEQ_LEN)

In [ ]:
dataloader = DataLoader(dataset=datagen,  
                        batch_size=BATCH_SIZE, 
                        num_workers=multiprocessing.cpu_count(), 
                        shuffle=SHUFFLE)

In [ ]:
model.cuda()

In [ ]:
# loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# initialize comet experiment
COMET_ML_EXPERIMENT, \
COMET_ML_EXPERIMENT_LINK = initialize_comet_ml_experiment(model_name=local_util.config.PROJECT_NAME, 
                                                          model_version='x.0.0')

In [ ]:
losses = list()
running_loss = 0.0
every = 200
for epoch in range(2):
    for i, x in tqdm_notebook(enumerate(dataloader), total=len(dataloader)):
        inputs = x['seq']
        labels = x['label'].float()

        inputs = inputs.cuda()
        labels = labels.cuda()

        #
        optimizer.zero_grad()

        # 
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if i % 2 == 0:
            print('loss:', loss)

        # print statistics
        running_loss += loss.item()
        if i % every == every - 1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / every))
            running_loss = 0.0

In [ ]:
import numpy as np
import torch

In [ ]:
emb_layer = torch.nn.Embedding(num_embeddings=17, embedding_dim=8)

In [ ]:
# emb_layer(torch.tensor([0, 0, 10, 16, 17]))